In [1]:
import numpy as np
import pandas as pd
import re
from convokit import Corpus, download
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score

In [2]:
utts = pd.read_csv('data/utts.csv')
roberts = pd.read_csv('data/subset.csv')

1024

In [3]:
#corpus = Corpus(filename=download('supreme-corpus'))
#corpus.load_info('utterance',['tokens']) 
"""
Karen recommended this over doing the Count Vectorization but I'm not yet clear
how this works, in progress
"""

"\nKaren recommended this over doing the Count Vectorization but I'm not yet clear\nhow this works, in progress\n"

In [4]:
"""
Everything below is for a Naive Bayes model which is distinctly 
not what we said but I wanted to get a general feel for building a model and things that we may want to tweak 
for our Bag of Words and beyond.
"""
utts['text'].replace('\d+', '', regex=True, inplace = True) #remove numbers, decide as group if this makes sense
group_utts = utts.groupby('meta.case_id')['text'].apply(' '.join)
#NOTE: No stemming or lemmatization done at this point
df = pd.merge(group_utts,roberts[['id','win_side']], how = 'left', 
              left_on = 'meta.case_id', right_on = 'id')

In [5]:
coun_vect = CountVectorizer(stop_words = 'english')
count_df = coun_vect.fit_transform(df['text'])
count_array = count_df.toarray()
count_df = pd.DataFrame(count_array,columns = coun_vect.get_feature_names())
print(count_df)

      aa  aaa  aagency  aaidd  aals  aamr  aaron  aarp  aau  aaup  ...  \
0      0    0        0      0     0     0      0     0    0     0  ...   
1      0    0        0      0     0     0      0     0    0     0  ...   
2      0    0        0      0     0     0      0     0    0     0  ...   
3      0    0        0      0     0     0      0     0    0     0  ...   
4      0    0        0      0     0     0      0     0    0     0  ...   
...   ..  ...      ...    ...   ...   ...    ...   ...  ...   ...  ...   
1019   0    0        0      0     0     0      0     0    0     0  ...   
1020   0    0        0      0     0     0      0     0    0     0  ...   
1021   0    0        0      0     0     0      0     0    0     0  ...   
1022   0    0        0      0     0     0      0     0    0     0  ...   
1023   0    0        0      0     0     0      0     0    0     0  ...   

      zooming  zuber  zubik  zubin  zug  zumudio  zuni  zurich  zurko  \
0           0      0      0      0    

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    count_df,roberts['win_side'], test_size = 0.2, random_state = 0)

In [7]:
model = GaussianNB() 
model.fit(X_train,y_train)
win_pred = model.predict(X_test)

In [8]:
print(accuracy_score(y_test,win_pred))
"""
this accuracy is notably worse than the 'just say the petitioner will win 
every time and you'll be right 65% of the time'
"""
print(precision_score(y_test,win_pred))

0.5902439024390244
0.6862745098039216
